In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine


db_username = 'myuser'
db_password = 'root'
db_host = 'localhost'
db_port = '5432'
db_name = 'medwell_db'

# Create a connection string and engine
connection_string = f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

In [2]:
user_id=9

In [3]:
query=f'''
select submitted_at,hemoglobin,rbc_count,wbc_count,platelet_count,pcv,bilirubin,proteins, calcium,blood_urea,sr_cholestrol
from patient_report pr join patient_reportdetail rd
on pr.id=rd.report_id
where user_id={user_id};
'''

In [4]:
df=pd.read_sql_query(query,engine)

In [5]:
df

,submitted_at,hemoglobin,rbc_count,wbc_count,platelet_count,pcv,bilirubin,proteins,calcium,blood_urea,sr_cholestrol
0,2024-10-22,12.1,4.72,9700,219000,38,0.57,6.18,8.19,21,-1
1,2024-10-22,12.1,4.72,9700,219000,38,0.57,6.18,8.19,21,-1


In [13]:
df.replace("-1",None,inplace=True)

In [14]:
df.to_dict("list")

{'submitted_at': [datetime.date(2024, 10, 22), datetime.date(2024, 10, 22)],
 'hemoglobin': ['12.1', '12.1'],
 'rbc_count': ['4.72', '4.72'],
 'wbc_count': ['9700', '9700'],
 'platelet_count': ['219000', '219000'],
 'pcv': ['38', '38'],
 'bilirubin': ['0.57', '0.57'],
 'proteins': ['6.18', '6.18'],
 'calcium': ['8.19', '8.19'],
 'blood_urea': ['21', '21'],
 'sr_cholestrol': [None, None]}

In [15]:
query=f'''
SELECT 
    AVG(CASE WHEN hemoglobin <> '-1' THEN CAST(hemoglobin AS FLOAT) END) AS avg_hemoglobin,
    AVG(CASE WHEN rbc_count <> '-1' THEN CAST(rbc_count AS FLOAT) END) AS avg_rbc_count,
    AVG(CASE WHEN wbc_count <> '-1' THEN CAST(wbc_count AS FLOAT) END) AS avg_wbc_count,
    AVG(CASE WHEN platelet_count <> '-1' THEN CAST(platelet_count AS FLOAT) END) AS avg_platelet_count,
    AVG(CASE WHEN pcv <> '-1' THEN CAST(pcv AS FLOAT) END) AS avg_pcv,
    AVG(CASE WHEN bilirubin <> '-1' THEN CAST(bilirubin AS FLOAT) END) AS avg_bilirubin,
    AVG(CASE WHEN proteins <> '-1' THEN CAST(proteins AS FLOAT) END) AS avg_proteins,
    AVG(CASE WHEN calcium <> '-1' THEN CAST(calcium AS FLOAT) END) AS avg_calcium,
    AVG(CASE WHEN blood_urea <> '-1' THEN CAST(blood_urea AS FLOAT) END) AS avg_blood_urea,
    AVG(CASE WHEN sr_cholestrol <> '-1' THEN CAST(sr_cholestrol AS FLOAT) END) AS avg_sr_cholestrol
FROM 
    patient_report pr 
JOIN 
    patient_reportdetail rd ON pr.id = rd.report_id
WHERE 
    user_id = {user_id};
'''

In [17]:
df=pd.read_sql_query(query,engine)

In [21]:
df.to_dict("records")[0]

{'avg_hemoglobin': 12.1,
 'avg_rbc_count': 4.72,
 'avg_wbc_count': 9700.0,
 'avg_platelet_count': 219000.0,
 'avg_pcv': 38.0,
 'avg_bilirubin': 0.57,
 'avg_proteins': 6.18,
 'avg_calcium': 8.19,
 'avg_blood_urea': 21.0,
 'avg_sr_cholestrol': None}

In [26]:
pd.read_sql_query(
    '''
    select count(*) as num_reports from patient_report
where user_id=9;
    ''',engine
)['num_reports'][0]

2

In [38]:
queries={
    'health_check':'''
    select submitted_at,hemoglobin,rbc_count,wbc_count,platelet_count,pcv,bilirubin,proteins, calcium,blood_urea,sr_cholestrol
    from patient_report pr join patient_reportdetail rd
    on pr.id=rd.report_id
    where user_id={user_id};
    ''',
    'average_query':'''
    SELECT 
        AVG(CASE WHEN hemoglobin <> '-1' THEN CAST(hemoglobin AS FLOAT) END) AS avg_hemoglobin,
        AVG(CASE WHEN rbc_count <> '-1' THEN CAST(rbc_count AS FLOAT) END) AS avg_rbc_count,
        AVG(CASE WHEN wbc_count <> '-1' THEN CAST(wbc_count AS FLOAT) END) AS avg_wbc_count,
        AVG(CASE WHEN platelet_count <> '-1' THEN CAST(platelet_count AS FLOAT) END) AS avg_platelet_count,
        AVG(CASE WHEN pcv <> '-1' THEN CAST(pcv AS FLOAT) END) AS avg_pcv,
        AVG(CASE WHEN bilirubin <> '-1' THEN CAST(bilirubin AS FLOAT) END) AS avg_bilirubin,
        AVG(CASE WHEN proteins <> '-1' THEN CAST(proteins AS FLOAT) END) AS avg_proteins,
        AVG(CASE WHEN calcium <> '-1' THEN CAST(calcium AS FLOAT) END) AS avg_calcium,
        AVG(CASE WHEN blood_urea <> '-1' THEN CAST(blood_urea AS FLOAT) END) AS avg_blood_urea,
        AVG(CASE WHEN sr_cholestrol <> '-1' THEN CAST(sr_cholestrol AS FLOAT) END) AS avg_sr_cholestrol
    FROM 
        patient_report pr 
    JOIN 
        patient_reportdetail rd ON pr.id = rd.report_id
    WHERE 
        user_id = {user_id};
    ''',
    'report_count_query': '''select count(*) as num_reports from patient_report where user_id={user_id}; '''
}


def provide_health_check_data(user_id):
    if pd.read_sql_query(queries['report_count_query'].format(user_id=user_id),engine)['num_reports'][0]<=3:
        return {"status":False}
    df=pd.read_sql_query(queries['health_check'].format(user_id=user_id),engine)
    df.replace("-1",None,inplace=True)
    data=df.to_dict("list")
    avg_data=pd.read_sql_query(queries['average_query'].format(user_id=user_id),engine).to_dict("records")[0]
    return {"status":True,"data":data,"avg_data":avg_data}

In [39]:
provide_health_check_data(9)

{'status': False}